## Imports

In [162]:
import pandas as pd
import numpy as np
import re
import os
import folium

## Data Read-in

In [163]:
df = pd.read_csv('redfin_2023-04-05-06-49-30.csv',dtype='str')

## Data Clean

In [164]:
df = df.rename(columns={'URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)':'URL'})

In [165]:
df = df.dropna(subset=['SOLD DATE'])

In [166]:
# Define list of desired months (excluding current month)
desired_months = ['February', 'March']

# Filter DataFrame to include only entries from desired months
df_filtered = df[df['SOLD DATE'].str.split('-', expand=True)[0].isin(desired_months)]

# Reset the index
df_filtered = df_filtered.reset_index(drop=True)

In [168]:
# Data checks
print(df_filtered['PRICE'].isna().value_counts())
print('-------')
print(df_filtered['$/SQUARE FEET'].isna().value_counts())
print('-------')
print(df_filtered['YEAR BUILT'].isna().value_counts())
print('-------')

False    1550
Name: PRICE, dtype: int64
-------
False    1546
True        4
Name: $/SQUARE FEET, dtype: int64
-------
False    1550
Name: YEAR BUILT, dtype: int64
-------


In [169]:
df_filtered.loc[df_filtered['PRICE'] == '0']

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE


In [170]:
df_filtered['PRICE'] = pd.to_numeric(df_filtered['PRICE'])
df_filtered['$/SQUARE FEET'] = pd.to_numeric(df_filtered['$/SQUARE FEET'])
df_filtered['YEAR BUILT'] = pd.to_numeric(df_filtered['YEAR BUILT'])
df_filtered['LATITUDE'] = pd.to_numeric(df_filtered['LATITUDE'])
df_filtered['LONGITUDE'] = pd.to_numeric(df_filtered['LONGITUDE'])

In [171]:
df_filtered.sort_values(by='PRICE',ascending=True).head(20)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
402,PAST SALE,March-24-2023,Condo/Co-op,4712 Lucerne Lakes Blvd #102,Lake Worth,FL,33467,220,2,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Lake-Worth/4712-Luce...,Beaches MLS,RX-10859670,N,Y,26.606876,-80.158146
1279,PAST SALE,March-29-2023,Condo/Co-op,801 Lake Shore Dr #718,Lake Park,FL,33403,395,2,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Lake-Park/801-Lake-S...,Beaches MLS,RX-10845757,N,Y,26.799882,-80.052684
1062,PAST SALE,March-1-2023,Condo/Co-op,836 E Tiffany Dr #4,Mangonia Park,FL,33407,54956,2,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Mangonia-Park/836-E-...,Beaches MLS,RX-10849930,N,Y,26.758292,-80.084081
273,PAST SALE,March-15-2023,Condo/Co-op,164 Cambridge G #164,West Palm Beach,FL,33417,55000,1,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/164-...,MARMLS,A11345268,N,Y,26.720288,-80.130766
1291,PAST SALE,February-24-2023,Condo/Co-op,37 Southport Ln Unit F,Boynton Beach,FL,33436,60000,2,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Boynton-Beach/37-Sou...,Beaches MLS,RX-10841228,N,Y,26.486121,-80.102507
1167,PAST SALE,March-31-2023,Condo/Co-op,101 Oxford 700,West Palm Beach,FL,33417,70000,1,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/101-...,Beaches MLS,RX-10853313,N,Y,26.718897,-80.127511
81,PAST SALE,February-15-2023,Condo/Co-op,5243 Cresthaven D Blvd Unit D,West Palm Beach,FL,33415,70000,1,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/5243...,Beaches MLS,RX-10858524,N,Y,26.637467,-80.123165
212,PAST SALE,March-6-2023,Condo/Co-op,38 Coventry B,West Palm Beach,FL,33417,70000,1,1.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/38-C...,Beaches MLS,RX-10860877,N,Y,26.718949,-80.122298
694,PAST SALE,February-28-2023,Condo/Co-op,225 Windsor Unit J,West Palm Beach,FL,33417,71000,1,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/225-...,MARMLS,A11319612,N,Y,26.710397,-80.134759
472,PAST SALE,February-23-2023,Condo/Co-op,124 Andover E #124,West Palm Beach,FL,33417,72500,1,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/124-...,Beaches MLS,RX-10859260,N,Y,26.710527,-80.128869


In [172]:
print(df_filtered['URL'].iloc[1279])

https://www.redfin.com/FL/Lake-Worth/4712-Lucerne-Lakes-Blvd-E-33467/unit-102/home/42180743


In [173]:
# Correct the prices, if needed
df_filtered.at[402,'PRICE']=(220000)
df_filtered.at[1279,'PRICE']=(220000)

In [174]:
# Find problem psf by searching for a '0' value
df_filtered.loc[df_filtered['$/SQUARE FEET'] == '0'][['SOLD DATE','ADDRESS','CITY','$/SQUARE FEET','PRICE','SQUARE FEET']]

,SOLD DATE,ADDRESS,CITY,$/SQUARE FEET,PRICE,SQUARE FEET


In [177]:
# Corrections, if needed
df_filtered.at[1279,'$/SQUARE FEET']=(395000/1316)
df_filtered.at[402,'$/SQUARE FEET']=(220000/1244)

In [178]:
# Find problem psf by searching for low values
df_filtered.sort_values(by='$/SQUARE FEET',ascending=True).head(20)[['ADDRESS','CITY','$/SQUARE FEET']]

,ADDRESS,CITY,$/SQUARE FEET
1291,37 Southport Ln Unit F,Boynton Beach,43.0
92,1532 Bridgewood Dr #1532,Boca Raton,60.0
1062,836 E Tiffany Dr #4,Mangonia Park,65.0
1434,25 Stratford Dr Unit A,Boynton Beach,81.0
850,105 E Tiffany Dr #2,Mangonia Park,82.0
881,9 Southport Ln E Unit E,Boynton Beach,83.0
793,723 Tiffany Dr W #4,West Palm Beach,89.0
158,2606 S Garden Dr #110,Lake Worth,93.0
1506,10172 Andover Coach Cir Unit H1,Lake Worth,94.0
76,312 NE 17th Ave #204,Boynton Beach,95.0


In [92]:
# print(df_filtered.URL.iloc[582])

In [93]:
# Corrections, if needed
#
#
#

In [94]:
# # Drop sales that aren't condos but labeled as such
# df_filtered = df_filtered.drop(1320)

## Make Maps

In [179]:
df_Current = df_filtered.loc[df_filtered['SOLD DATE'].str.contains('March')]

In [180]:
### Create a price column formatted as currency ###
df_Current['PRICE_AS_CURRENCY'] = df_Current['PRICE'].apply(lambda x: "${:,.0f}".format(x))
### Set formatting for Beds, Baths ###
df_Current['YEAR BUILT'] = df_Current['YEAR BUILT'].apply(lambda x: '{:.0f}'.format(x))
df_Current['PRICE_SQUARE_FEET_AS_CURRENCY'] = df_Current['$/SQUARE FEET'].apply(lambda x: '${:,.0f}'.format(x))

/var/folders/rq/0_j_rksd7psgdycg9c1qwf6m0000gp/T/ipykernel_58316/21604551.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_Current['PRICE_AS_CURRENCY'] = df_Current['PRICE'].apply(lambda x: "${:,.0f}".format(x))
/var/folders/rq/0_j_rksd7psgdycg9c1qwf6m0000gp/T/ipykernel_58316/21604551.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_Current['YEAR BUILT'] = df_Current['YEAR BUILT'].apply(lambda x: '{:.0f}'.format(x))
/var/folders/rq/0_j_rksd7psgdycg9c1qwf6m0000gp/T/ipykernel_58316/21604551.py:5: 

In [181]:
df_Current = df_Current.sort_values(by=['PRICE'], ascending=False)
### Insert different colors for top 10 sales vs. the rest ###
df_Current['COLOR'] = ''
### Create RANK column ###
df_Current['RANK'] = 0
### Insert RANK values ###
df_Current['RANK'] = range(1, len(df_Current) + 1)
# use numpy to assign values to the 'COLOR' column
df_Current['COLOR'] = np.where(df_Current['RANK'] <= 10, 'orange', 'blue')

## HTML Popup Formatter

In [182]:
### Define list of columns to drop from DF ###
columns_drop = ['SALE TYPE','PROPERTY TYPE','STATE OR PROVINCE','ZIP OR POSTAL CODE','HOA/MONTH','STATUS','NEXT OPEN HOUSE START TIME','NEXT OPEN HOUSE END TIME','SOURCE','MLS#','FAVORITE','INTERESTED','SQUARE FEET','LOT SIZE']

In [183]:
### Drop the columns ###
df_Current = df_Current.drop(columns=columns_drop)

In [184]:
def popup_html(row):
    Price = row['PRICE_AS_CURRENCY']
    Address = row['ADDRESS']
    City = row['CITY']
    sold_date = row['SOLD DATE']
    beds = row['BEDS']
    baths = row['BATHS']
    psf = row['PRICE_SQUARE_FEET_AS_CURRENCY']
    year_built = row['YEAR BUILT']
    rank = row['RANK']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Price: </strong>{}'''.format(Price) + '''<br>
    <strong>Address: </strong>{}'''.format(Address) + '''<br>
    <strong>City: </strong>{}'''.format(City) + '''<br>
    <strong>Sold: </strong>{}'''.format(sold_date) + '''<br>
    <strong>Beds: </strong>{}'''.format(beds) + '''<br>
    <strong>Baths: </strong>{}'''.format(baths) + '''<br>
    <strong>Price per sf: </strong>{}'''.format(psf) + '''<br>
    <strong>Year Built: </strong>{}'''.format(year_built) + '''<br>
    <strong>Price Rank: </strong>{}'''.format(rank) + '''
    </html>
    '''
    return html

In [187]:
### Create map container ###
m = folium.Map(location=df_Current[["LATITUDE", "LONGITUDE"]].mean().to_list(),zoom_start=10.5,tiles=None)

### Create title ###
title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f"March 2023 Condo Sales")

m.get_root().html.add_child(folium.Element(title_html))

# Create two FeatureGroups for different color pins
fg_blue = folium.FeatureGroup(name='All other sales')
fg_orange = folium.FeatureGroup(name='Top 10 Sales')

for index, row in df_Current.iterrows():
    # Add the markers to the appropriate FeatureGroup based on the color
    if row['COLOR'] == 'blue':
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_blue)
    else:
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_orange)

# Add the FeatureGroups to the map
fg_orange.add_to(m)
fg_blue.add_to(m)

folium.TileLayer('OpenStreetMap',control=False).add_to(m)

# Add LayerControl to the map
folium.map.LayerControl(collapsed=False).add_to(m)

# Display map

In [188]:
m.save('index.html')

## Summary Info

In [189]:
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

Newest = '\033[1m' + 'Newest' + '\033[0m'
Oldest = '\033[1m' + 'Oldest' + '\033[0m'

In [190]:
df_Current.columns

Index(['SOLD DATE', 'ADDRESS', 'CITY', 'PRICE', 'BEDS', 'BATHS', 'LOCATION',
       'YEAR BUILT', 'DAYS ON MARKET', '$/SQUARE FEET', 'URL', 'LATITUDE',
       'LONGITUDE', 'PRICE_AS_CURRENCY', 'PRICE_SQUARE_FEET_AS_CURRENCY',
       'COLOR', 'RANK'],
      dtype='object')

In [191]:
df_Current['FULL_ADDRESS'] = df_Current['ADDRESS'] + ' ' + df_Current['CITY']

In [192]:
df_Current['YEAR BUILT'] = pd.to_numeric(df['YEAR BUILT'])

In [193]:
print(f"{ME}{BR}{df_Current.loc[df_Current['PRICE'].idxmax()]['LOCATION']}, {df_Current.loc[df_Current['PRICE'].idxmax()]['FULL_ADDRESS']} | Price ${df_Current.loc[df_Current['PRICE'].idxmax()]['PRICE']:,.0f} | ${df_Current.loc[df_Current['PRICE'].idxmax()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_Current.loc[df_Current['PRICE'].idxmax()]['YEAR BUILT']}")
print(f"{LE}{BR}{df_Current.loc[df_Current['PRICE'].idxmin()]['LOCATION']}, {df_Current.loc[df_Current['PRICE'].idxmin()]['FULL_ADDRESS']} | Price ${df_Current.loc[df_Current['PRICE'].idxmin()]['PRICE']:,.0f} | ${df_Current.loc[df_Current['PRICE'].idxmin()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_Current.loc[df_Current['PRICE'].idxmin()]['YEAR BUILT']}")

print(f"{MAX_PSF}{BR}{df_Current.loc[df_Current['$/SQUARE FEET'].idxmax()]['LOCATION']}, {df_Current.loc[df_Current['$/SQUARE FEET'].idxmax()]['FULL_ADDRESS']} | Price ${df_Current.loc[df_Current['$/SQUARE FEET'].idxmax()]['PRICE']:,.0f} | ${df_Current.loc[df_Current['$/SQUARE FEET'].idxmax()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_Current.loc[df_Current['$/SQUARE FEET'].idxmax()]['YEAR BUILT']}")
print(f"{MIN_PSF}{BR}{df_Current.loc[df_Current['$/SQUARE FEET'].idxmin()]['LOCATION']}, {df_Current.loc[df_Current['$/SQUARE FEET'].idxmin()]['FULL_ADDRESS']} | Price ${df_Current.loc[df_Current['$/SQUARE FEET'].idxmin()]['PRICE']:,.0f} | ${df_Current.loc[df_Current['$/SQUARE FEET'].idxmin()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_Current.loc[df_Current['$/SQUARE FEET'].idxmin()]['YEAR BUILT']}")

print(f"{Newest}{BR}{df_Current.loc[df_Current['YEAR BUILT'].idxmax()]['LOCATION']}, {df_Current.loc[df_Current['YEAR BUILT'].idxmax()]['FULL_ADDRESS']} | Price ${df_Current.loc[df_Current['YEAR BUILT'].idxmax()]['PRICE']:,.0f} | ${df_Current.loc[df_Current['YEAR BUILT'].idxmax()]['$/SQUARE FEET']} psf | Year built: {df_Current.loc[df_Current['YEAR BUILT'].idxmax()]['YEAR BUILT']}")
print(f"{Oldest}{BR}{df_Current.loc[df_Current['YEAR BUILT'].idxmin()]['LOCATION']}, {df_Current.loc[df_Current['YEAR BUILT'].idxmin()]['FULL_ADDRESS']} | Price ${df_Current.loc[df_Current['YEAR BUILT'].idxmin()]['PRICE']:,.0f} | ${df_Current.loc[df_Current['YEAR BUILT'].idxmin()]['$/SQUARE FEET']} psf | Year built: {df_Current.loc[df_Current['YEAR BUILT'].idxmin()]['YEAR BUILT']}")

Most Expensive
Seaglass Condominium, 1500 Beach Rd #601 Tequesta | Price $10,650,000 | $2,964 psf | Year built: 1988.0
Least Expensive
Tiffany Lake Condo PH 3, 836 E Tiffany Dr #4 Mangonia Park | Price $54,956 | $65 psf | Year built: 1992.0
Highest Price Per Square Foot
Dunster House Condo, 360 S Ocean Blvd Unit 4c Palm Beach | Price $7,650,000 | $3,122 psf | Year built: 1992.0
Lowest Price Per Square Foot
BRIDGEWOOD MID-RISE CONDO, 1532 Bridgewood Dr #1532 Boca Raton | Price $75,000 | $60 psf | Year built: nan
Newest
Crestwood Square, 1500 S Crestwood Ct #1505 Royal Palm Beach | Price $285,000 | $229.0 psf | Year built: 2021.0
Oldest
Greentree Villas Condo, 4767 Greentree Cres Unit A Boynton Beach | Price $265,000 | $189.0 psf | Year built: 1925.0


## Time on Market Calculator

In [415]:
# print(df_Current.loc[df_Current['YEAR BUILT'].idxmin()])

In [416]:
from datetime import datetime, timedelta

date1 = datetime(2022, 12, 6) ## List (Earlier) date
date2 = datetime(2023, 3, 24) ## Close (Later) date

delta = date2 - date1
num_days = delta.days

print(num_days)

108


## Map URL Snagger

In [194]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

In [195]:
cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/Palm_Beach_condo_sales_month_ending_march_2023


## Get Summary Data

In [196]:
print('SALES INFO')
print(f'Number of sales: {len(df_Current)}')
print('--------')
print(f'Total sale price: ${df_Current["PRICE"].sum():,.0f}')
print('--------')
print(f'Median sale price: ${df_Current["PRICE"].median():,.0f}')
print('--------')
print(f'Max sale price: ${df_Current["PRICE"].max():,.0f}')
print('--------')
print(f'Min sale price: ${df_Current["PRICE"].min():,.0f}')
print('------------------------------------------------')
print('PSF INFO')
print(f'Max price per square foot: ${df_Current["$/SQUARE FEET"].max():,.0f}')
print('--------')
print(f'Min price per square foot: ${df_Current["$/SQUARE FEET"].min():,.0f}')
print('--------')
print(f'Median price per square foot: ${df_Current["$/SQUARE FEET"].median():,.0f}')
print('------------------------------------------------')
print('CONDO AGES')
print(f'Newest building: {df_Current["YEAR BUILT"].max()}')
print('----------')
print(f'Oldest building: {df_Current["YEAR BUILT"].min()}')
print('----------')
print(f'Average building age: {df_Current["YEAR BUILT"].mean()}')

SALES INFO
Number of sales: 942
--------
Total sale price: $460,616,290
--------
Median sale price: $265,000
--------
Max sale price: $10,650,000
--------
Min sale price: $54,956
------------------------------------------------
PSF INFO
Max price per square foot: $3,122
--------
Min price per square foot: $60
--------
Median price per square foot: $232
------------------------------------------------
CONDO AGES
Newest building: 2021.0
----------
Oldest building: 1925.0
----------
Average building age: 1982.6240409207162


In [197]:
df_filtered

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
0,PAST SALE,March-20-2023,Condo/Co-op,250 Tresana Blvd #86,Jupiter,FL,33478,1200000,3,2.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Jupiter/250-Tresana-...,Beaches MLS,RX-10859283,N,Y,26.927457,-80.155025
1,PAST SALE,March-20-2023,Condo/Co-op,3755 Via Poinciana #513,Lake Worth,FL,33467,152000,2,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Lake-Worth/3755-Via-...,Beaches MLS,RX-10867629,N,Y,26.621251,-80.154440
2,PAST SALE,March-16-2023,Condo/Co-op,33 Colonial Club Dr #105,Boynton Beach,FL,33435,250000,2,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Boynton-Beach/33-Col...,Beaches MLS,RX-10851680,N,Y,26.500290,-80.059495
3,PAST SALE,March-31-2023,Condo/Co-op,14484 Canalview Dr Unit C,Delray Beach,FL,33484,160000,1,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Delray-Beach/14484-C...,Beaches MLS,RX-10861599,N,Y,26.463115,-80.125594
4,PAST SALE,March-27-2023,Condo/Co-op,806 E Windward Way #506,Lantana,FL,33462,438000,2,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Lantana/806-E-Windwa...,Beaches MLS,RX-10833909,N,Y,26.591130,-80.049909
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1545,PAST SALE,March-24-2023,Condo/Co-op,646 Snug Harbor Dr Unit H208,Boynton Beach,FL,33435,285000,2,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Boynton-Beach/646-Sn...,Beaches MLS,RX-10793995,N,Y,26.516133,-80.055982
1546,PAST SALE,March-17-2023,Condo/Co-op,4051 Dale Rd,Palm Springs,FL,33406,406500,4,3.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/4051...,Beaches MLS,RX-10792072,N,Y,26.638670,-80.105414
1547,PAST SALE,March-16-2023,Condo/Co-op,404 Normandy I #404,Delray Beach,FL,33484,205000,2,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Delray-Beach/404-Nor...,MARMLS,A11180115,N,Y,26.445317,-80.138579
1548,PAST SALE,March-21-2023,Condo/Co-op,5500 NW 2nd Ave #518,Boca Raton,FL,33487,220000,1,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/5500-NW-2...,Beaches MLS,F10323573,N,Y,26.398361,-80.083044
